# Using fine-tuned models
---
This notebook is intended to serve as an example of how to communicate and use previously fine-tuned models.The different model names are stored in Models.jsonl.

#### Imports

In [4]:
import openai
from openai import OpenAI
import os
import time
import json

import random
import re
import pickle
import scapy

from scapy.all import *
from scapy.utils import RawPcapReader, wrpcap
import scapy.all as scapy
from scapy.layers.l2 import Ether, ARP
from scapy.layers.inet import IP, ICMP, TCP
from scapy.layers.http import *

#### Definitions

In [ ]:
os.environ['OPENAI_API_KEY'] = 'FILL'
client = OpenAI()

tipo ="Conversations"
wanted_Model = "DNS"
Models_file = 'Models.jsonl'

dataset = "../../data/Conversations/DNS/DNS_Conv_Testing_v2.txt"
pkl_saved = "../../data/Fine_Tuned/DNS/pickle/Test_DNS_CONV_Fine_Tuned_v3.pkl"
pcap_saved = "../../data/Fine_Tuned/DNS/pcap/Test_DNS_CONV_v2_generated_3_5_turbo-instruct_conv_v3.pcap"

system_message = 'You are a new generation traffic generator. \
You are specilized in the DNS protocol and conversations generation using python and scapy. \
You are especially attentive to variables and different types of traffic.'

### Fine-Tuned models list

In [ ]:
with open(Models_file, 'r') as json_file:
    Models_list = json.load(json_file)

print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
print("                                   Paquetes                                   ")
print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++\n\n")
print("----------Lista de modelos de paquetes----------")
for key, value in Models_list["Packets"]["Specific"].items():
    print( "\t" + key + " -> " + value)

    
print("----------Problemas de los modelos de paquetes----------")
for key, value in Models_list["Packets"]["Problems"].items():
    print( "\t" + key + " -> " + value)

print("\n\n\n++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
print("                               Conversaciones                                 ")
print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++\n\n")
print("----------Lista de modelos de Conversaciones----------")
for key, value in Models_list["Conversations"]["Specific"].items():
    print( "\t" + key + " -> " + value)

    
print("----------Problemas de los modelos de Conversaciones----------")
for key, value in Models_list["Conversations"]["Problems"].items():
    print( "\t" + key + " -> " + value)

### Loading the query dataset

In [37]:
with open(dataset,"r") as f:
    packets_summary = f.read().splitlines()
len(packets_summary)

220

### Loading a pickle file to save the responses 

In [38]:
class response():
    "Stores name and place pairs"
    def __init__(self, name, place):
        self.prompt_summary = name
        self.completion = place

try:
    responses = pickle.load( open(pkl_saved, "rb" ))
    print("Already" + str(len(responses)) + " generated.")
except:
    responses = []
    print("No responses previously generated")

No responses previously generated


### Packet command generation using the fine tuned model

In [39]:
for index in range(200):
    
    summary = packets_summary[index]
    print("...........................................")
    print( " GENERATING SCAPY COMMANDS FOR THE FOLLOWING PACKETS (iter " +str(index+1) +")")
    
    
    #summaries = random.choices(packets_summary,k=summaries_per_request)
    
    messages=[{"role": "system", "content": system_message},
        {"role": "user", "content": summary}]
    
    print("\n This is the message sent: ")
    print('\t"role": "system", "content" : \n\t\t' + system_message)
    print('\n\t"role": "user", "content" : ' + summary)
    #for summary in summaries:
    #    print("\t\t" + summary)
    
    #Esta es la petición que le mandas al modelo de IA
    completion = client.chat.completions.create(
        model=Models_list[tipo]["Specific"][wanted_Model], 
        messages=messages,
        max_tokens=2600,
        temperature=0.1)

    
    print("...Generated!")
    
    # Guardamos tanto el resumen de los paquetes que solicitamos y lo que devuelve el modelo para estos paquetes
    responses.append(response(messages, completion))
    
    print("The response is: ")
    print(responses[-1].completion.choices[0].message.content)
    
    print("\nNumber of completitions done: " + str(len(responses)))
    
    exec(responses[-1].completion.choices[0].message.content)
    print("\n")
    
    pkt_list = [pkt_1, pkt_2, pkt_3, pkt_4, pkt_5, pkt_6, pkt_7, pkt_8, pkt_9, pkt_10, pkt_11]
    
    # Guardamos los paquetes generados en un pcap
    with open(pcap_saved, "ba+") as f:
        wrpcap(f, pkt_list, append=True)
        
    # Guardamos tanto la petición como la respuesta en un archivo
    pickle.dump(responses, open(pkl_saved, "wb" ) )

...........................................
 GENERATING SCAPY COMMANDS FOR THE FOLLOWING PACKETS (iter 1)

 This is the message sent: 
	"role": "system", "content" : 
		You are a new generation traffic generator. You are specilized in the DNS protocol and conversations generation using python and scapy. You are especially attentive to variables and different types of traffic.

	"role": "user", "content" : Source: IP= "203.0.113.5" // Destination: IP= "203.0.113.15" // Others: id=0x1234 , resource= "www.spotify.es" , response = "104.154.126.229"
...Generated!
The response is: 

RANDOM_PORT = random.randint(4097, 65530)
pkt_1 = IP(src="203.0.113.5", dst="203.0.113.15")/UDP(sport=RANDOM_PORT, dport=53)/DNS(id=0x1234, qr=0, rd=1, opcode=0, qdcount=1, ancount=0, nscount=0, arcount=0, qd=DNSQR(qname="www.spotify.es", qtype="A", qclass="IN"))
time.sleep(abs(random.gauss(0, 0.03)))
pkt_2 = IP(src="203.0.113.15", dst="203.0.113.5")/UDP(sport=53, dport=RANDOM_PORT)/DNS(id=0x0aba, qr=1, opcode=0,

...Generated!
The response is: 

RANDOM_PORT = random.randint(4097, 65530)
pkt_1 = IP(src="198.51.100.15", dst="198.51.100.25")/UDP(sport=RANDOM_PORT, dport=53)/DNS(id=0x8901, qr=0, rd=1, opcode=0, qdcount=1, ancount=0, nscount=0, arcount=0, qd=DNSQR(qname="www.booking.com", qtype="A", qclass="IN"))
time.sleep(abs(random.gauss(0, 0.03)))
pkt_2 = IP(src="198.51.100.25", dst="198.51.100.15")/UDP(sport=53, dport=RANDOM_PORT)/DNS(id=0x0aba, qr=1, opcode=0, ra=1, rcode=0, qdcount=1, ancount=1, nscount=0, arcount=0, qd=DNSQR(qname="www.booking.com", qtype="A", qclass="IN"), an=DNSRR(rrname="www.booking.com", type="A", rclass="IN", ttl=255, rdata="5.57.16.220"))

Number of completitions done: 8


...........................................
 GENERATING SCAPY COMMANDS FOR THE FOLLOWING PACKETS (iter 9)

 This is the message sent: 
	"role": "system", "content" : 
		You are a new generation traffic generator. You are specilized in the DNS protocol and conversations generation using python and sca

...Generated!
The response is: 

RANDOM_PORT = random.randint(4097, 65530)
pkt_1 = IP(src="203.0.113.51", dst="203.0.113.61")/UDP(sport=RANDOM_PORT, dport=53)/DNS(id=0x5679, qr=0, rd=1, opcode=0, qdcount=1, ancount=0, nscount=0, arcount=0, qd=DNSQR(qname="www.hotels.com", qtype="A", qclass="IN"))
time.sleep(abs(random.gauss(0, 0.03)))
pkt_2 = IP(src="203.0.113.61", dst="203.0.113.51")/UDP(sport=53, dport=RANDOM_PORT)/DNS(id=0x0aba, qr=1, opcode=0, ra=1, rcode=0, qdcount=1, ancount=1, nscount=0, arcount=0, qd=DNSQR(qname="www.hotels.com", qtype="A", qclass="IN"), an=DNSRR(rrname="www.hotels.com", type="A", rclass="IN", ttl=255, rdata="23.3.123.107"))

Number of completitions done: 15


...........................................
 GENERATING SCAPY COMMANDS FOR THE FOLLOWING PACKETS (iter 16)

 This is the message sent: 
	"role": "system", "content" : 
		You are a new generation traffic generator. You are specilized in the DNS protocol and conversations generation using python and scapy. 

...Generated!
The response is: 

RANDOM_PORT = random.randint(4097, 65530)
pkt_1 = IP(src="37.3.7.15", dst="37.4.12.18")/UDP(sport=RANDOM_PORT, dport=53)/DNS(id=0x1002, qr=0, rd=1, opcode=0, qdcount=1, ancount=0, nscount=0, arcount=0, qd=DNSQR(qname="www.company2.com", qtype="A", qclass="IN"))
time.sleep(abs(random.gauss(0, 0.03)))
pkt_2 = IP(src="37.4.12.18", dst="37.3.7.15")/UDP(sport=53, dport=RANDOM_PORT)/DNS(id=0x0aba, qr=1, opcode=0, ra=1, rcode=0, qdcount=1, ancount=1, nscount=0, arcount=0, qd=DNSQR(qname="www.company2.com", qtype="A", qclass="IN"), an=DNSRR(rrname="www.company2.com", type="A", rclass="IN", ttl=255, rdata="140.4.6.10"))

Number of completitions done: 22


...........................................
 GENERATING SCAPY COMMANDS FOR THE FOLLOWING PACKETS (iter 23)

 This is the message sent: 
	"role": "system", "content" : 
		You are a new generation traffic generator. You are specilized in the DNS protocol and conversations generation using python and scapy. You ar

...Generated!
The response is: 

RANDOM_PORT = random.randint(4097, 65530)
pkt_1 = IP(src="37.10.28.64", dst="37.11.47.67")/UDP(sport=RANDOM_PORT, dport=53)/DNS(id=0x1009, qr=0, rd=1, opcode=0, qdcount=1, ancount=0, nscount=0, arcount=0, qd=DNSQR(qname="www.company9.es", qtype="A", qclass="IN"))
time.sleep(abs(random.gauss(0, 0.03)))
pkt_2 = IP(src="37.11.47.67", dst="37.10.28.64")/UDP(sport=53, dport=RANDOM_PORT)/DNS(id=0x0aba, qr=1, opcode=0, ra=1, rcode=0, qdcount=1, ancount=1, nscount=0, arcount=0, qd=DNSQR(qname="www.company9.es", qtype="A", qclass="IN"), an=DNSRR(rrname="www.company9.es", type="A", rclass="IN", ttl=255, rdata="184.18.27.45"))

Number of completitions done: 29


...........................................
 GENERATING SCAPY COMMANDS FOR THE FOLLOWING PACKETS (iter 30)

 This is the message sent: 
	"role": "system", "content" : 
		You are a new generation traffic generator. You are specilized in the DNS protocol and conversations generation using python and scapy. Y

...

...Generated!
The response is: 

RANDOM_PORT = random.randint(4097, 65530)
pkt_1 = IP(src="37.150.195.147", dst="37.151.235.150")/UDP(sport=RANDOM_PORT, dport=53)/DNS(id=0x1095, qr=0, rd=1, opcode=0, qdcount=1, ancount=0, nscount=0, arcount=0, qd=DNSQR(qname="www.company149.es", qtype="A", qclass="IN"))
time.sleep(abs(random.gauss(0, 0.03)))
pkt_2 = IP(src="37.151.235.150", dst="37.150.195.147")/UDP(sport=53, dport=RANDOM_PORT)/DNS(id=0x0aba, qr=1, opcode=0, ra=1, rcode=0, qdcount=1, ancount=1, nscount=0, arcount=0, qd=DNSQR(qname="www.company149.es", qtype="A", qclass="IN"), an=DNSRR(rrname="www.company149.es", type="A", rclass="IN", ttl=255, rdata="184.46.195.235"))

Number of completitions done: 169


...........................................
 GENERATING SCAPY COMMANDS FOR THE FOLLOWING PACKETS (iter 170)

 This is the message sent: 
	"role": "system", "content" : 
		You are a new generation traffic generator. You are specilized in the DNS protocol and conversations generation usi

...Generated!
The response is: 

RANDOM_PORT = random.randint(4097, 65530)
pkt_1 = IP(src="37.157.216.189", dst="37.158.14.192")/UDP(sport=RANDOM_PORT, dport=53)/DNS(id=0x109c, qr=0, rd=1, opcode=0, qdcount=1, ancount=0, nscount=0, arcount=0, qd=DNSQR(qname="www.company156.net", qtype="A", qclass="IN"))
time.sleep(abs(random.gauss(0, 0.03)))
pkt_2 = IP(src="37.158.14.192", dst="37.157.216.189")/UDP(sport=53, dport=RANDOM_PORT)/DNS(id=0x0aba, qr=1, opcode=0, ra=1, rcode=0, qdcount=1, ancount=1, nscount=0, arcount=0, qd=DNSQR(qname="www.company156.net", qtype="A", qclass="IN"), an=DNSRR(rrname="www.company156.net", type="A", rclass="IN", ttl=255, rdata="23.60.216.14"))

Number of completitions done: 176


...........................................
 GENERATING SCAPY COMMANDS FOR THE FOLLOWING PACKETS (iter 177)

 This is the message sent: 
	"role": "system", "content" : 
		You are a new generation traffic generator. You are specilized in the DNS protocol and conversations generation usin

...Generated!
The response is: 

RANDOM_PORT = random.randint(4097, 65530)
pkt_1 = IP(src="37.164.237.231", dst="37.165.49.234")/UDP(sport=RANDOM_PORT, dport=53)/DNS(id=0x10a3, qr=0, rd=1, opcode=0, qdcount=1, ancount=0, nscount=0, arcount=0, qd=DNSQR(qname="www.company163.io", qtype="A", qclass="IN"))
time.sleep(abs(random.gauss(0, 0.03)))
pkt_2 = IP(src="37.165.49.234", dst="37.164.237.231")/UDP(sport=53, dport=RANDOM_PORT)/DNS(id=0x0aba, qr=1, opcode=0, ra=1, rcode=0, qdcount=1, ancount=1, nscount=0, arcount=0, qd=DNSQR(qname="www.company163.io", qtype="A", qclass="IN"), an=DNSRR(rrname="www.company163.io", type="A", rclass="IN", ttl=255, rdata="151.74.237.49"))

Number of completitions done: 183


...........................................
 GENERATING SCAPY COMMANDS FOR THE FOLLOWING PACKETS (iter 184)

 This is the message sent: 
	"role": "system", "content" : 
		You are a new generation traffic generator. You are specilized in the DNS protocol and conversations generation using 

...Generated!
The response is: 

RANDOM_PORT = random.randint(4097, 65530)
pkt_1 = IP(src="37.171.2.21", dst="37.172.84.24")/UDP(sport=RANDOM_PORT, dport=53)/DNS(id=0x10aa, qr=0, rd=1, opcode=0, qdcount=1, ancount=0, nscount=0, arcount=0, qd=DNSQR(qname="www.company170.org", qtype="A", qclass="IN"))
time.sleep(abs(random.gauss(0, 0.03)))
pkt_2 = IP(src="37.172.84.24", dst="37.171.2.21")/UDP(sport=53, dport=RANDOM_PORT)/DNS(id=0x0aba, qr=1, opcode=0, ra=1, rcode=0, qdcount=1, ancount=1, nscount=0, arcount=0, qd=DNSQR(qname="www.company170.org", qtype="A", qclass="IN"), an=DNSRR(rrname="www.company170.org", type="A", rclass="IN", ttl=255, rdata="206.88.2.84"))

Number of completitions done: 190


...........................................
 GENERATING SCAPY COMMANDS FOR THE FOLLOWING PACKETS (iter 191)

 This is the message sent: 
	"role": "system", "content" : 
		You are a new generation traffic generator. You are specilized in the DNS protocol and conversations generation using python 

...Generated!
The response is: 

RANDOM_PORT = random.randint(4097, 65530)
pkt_1 = IP(src="37.178.23.63", dst="37.179.119.66")/UDP(sport=RANDOM_PORT, dport=53)/DNS(id=0x10b1, qr=0, rd=1, opcode=0, qdcount=1, ancount=0, nscount=0, arcount=0, qd=DNSQR(qname="www.company177.com", qtype="A", qclass="IN"))
time.sleep(abs(random.gauss(0, 0.03)))
pkt_2 = IP(src="37.179.119.66", dst="37.178.23.63")/UDP(sport=53, dport=RANDOM_PORT)/DNS(id=0x0aba, qr=1, opcode=0, ra=1, rcode=0, qdcount=1, ancount=1, nscount=0, arcount=0, qd=DNSQR(qname="www.company177.com", qtype="A", qclass="IN"), an=DNSRR(rrname="www.company177.com", type="A", rclass="IN", ttl=255, rdata="5.102.23.119"))

Number of completitions done: 197


...........................................
 GENERATING SCAPY COMMANDS FOR THE FOLLOWING PACKETS (iter 198)

 This is the message sent: 
	"role": "system", "content" : 
		You are a new generation traffic generator. You are specilized in the DNS protocol and conversations generation using py

In [29]:
print(responses[-1].completion.choices[0].message.content)

scapy.Ether(src="c2:b3:a4:f5'e6'd7", dst="c2:b3:a4:f5:e6:d8")/scapy.ARP(op=2, psrc="172.16.0.11", hwsrc="c2:b3:a4:f5:e6:d7", hwdst="c2:b3:a4:f5:e6:d8",  pdst=socket.inet_ntoa(struct.pack(">I", random.randint(1, 0xffffffff))))


### Packet generation

In [39]:
# Ejecutamos los comandos que devuelve el modelo para generar los paquetes
exec("pkt= " + str(responses[-1].completion.choices[0].message.content))
print(pkt)

b'E\x00\x00\x1c\x00\x01\x00\x006\x01\xb2\x80\x05#\xe2\x88\xc0\xa8*\x0c\x00\x00\xfa\x98\x04V\x01\x11'


In [43]:
#Enviarlos para probar si se envian correctamente
send(pkt)


Sent 1 packets.


### Introduction to a pcap

In [46]:
# Guardamos los paquetes generados en un pcap
with open(pcap_saved, "ba+") as f:
    wrpcap(f, pkt, append=True)